In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [4]:
movies_data = pd.read_csv('combine_df.csv')
movies_data.head()

,budget,genres,movie_id,keywords,original_title,overview,popularity,production_companies,production_countries,release_date,...,tagline,vote_average,vote_count,cast,crew,cast_1,cast_2,genres_1,production_companies_1,release_year
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,...,Enter the World of Pandora.,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Sam Worthington,Zoe Saldana,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['Ingenious Film Partners', 'Twentieth Century...",2009.0
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,...,"At the end of the world, the adventure begins.",6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Johnny Depp,Orlando Bloom,"['Adventure', 'Fantasy', 'Action']","['Walt Disney Pictures', 'Jerry Bruckheimer Fi...",2007.0
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,...,A Plan No One Escapes,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",Daniel Craig,Christoph Waltz,"['Action', 'Adventure', 'Crime']","['Columbia Pictures', 'Danjaq', 'B24']",2015.0
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,...,The Legend Ends,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Christian Bale,Michael Caine,"['Action', 'Crime', 'Drama', 'Thriller']","['Legendary Pictures', 'Warner Bros.', 'DC Ent...",2012.0
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,...,"Lost in our world, found in another.",6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",Taylor Kitsch,Lynn Collins,"['Action', 'Adventure', 'Science Fiction']",['Walt Disney Pictures'],2012.0


## Weighted Rating
### ((v/v+m)*R) + ((m/v+m)*C)
#### V is the number of votes for the movie
#### m is the minimum votes required to be listed on chart
#### R is the average rating of the movie
#### C is mean vote across whole report

In [15]:
m = np.round(movies_data['vote_count'].quantile(0.90))
c = movies_data['vote_average'].mean()
print("m : {} , c : {}".format(m,c))

m : 1838.0 , c : 6.092171559442011


In [19]:
# Take only those values where votes are more than 1838
movies_data_1 = movies_data[movies_data['vote_count']> m][['vote_average', 'vote_count','original_title']]
movies_data_1

,vote_average,vote_count,original_title
0,7.2,11800,Avatar
1,6.9,4500,Pirates of the Caribbean: At World's End
2,6.3,4466,Spectre
3,7.6,9106,The Dark Knight Rises
4,6.1,2124,John Carter
...,...,...,...
4291,7.2,2184,Saw
4300,8.0,3697,Reservoir Dogs
4302,8.1,2311,"Il buono, il brutto, il cattivo"
4337,8.0,2535,Taxi Driver


In [20]:
# Calculate Weighted Rating 
def calculate_weighted_rating(x):
    v = x['vote_count']
    r = x ['vote_average']
    return ((v/(v+m))*r + (m/(v+m))*c)

movies_data_1['Weighted_rating'] = movies_data_1.apply(lambda x : calculate_weighted_rating(x), axis=1)    

In [25]:
top_weighted_rated_movies = movies_data_1.sort_values(by='Weighted_rating', ascending=False).head(10).reset_index(drop=True)
top_weighted_rated_movies

,vote_average,vote_count,original_title,Weighted_rating
0,8.5,8205,The Shawshank Redemption,8.059336
1,8.3,9413,Fight Club,7.939322
2,8.2,12002,The Dark Knight,7.920073
3,8.3,8428,Pulp Fiction,7.904716
4,8.1,13752,Inception,7.863285
5,8.4,5893,The Godfather,7.851327
6,8.1,10867,Interstellar,7.809533
7,8.2,7927,Forrest Gump,7.803258
8,8.1,8064,The Lord of the Rings: The Return of the King,7.727309
9,8.2,5879,The Empire Strikes Back,7.697967
